# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 51 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.00649


extracting tarball to tmp_2211.00649...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.00650


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.00649/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

/tmp/ipykernel_2267/4030337529.py:34: LatexWarning: 2211.01067 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2211.01172


extracting tarball to tmp_2211.01172... done.
Retrieving document from  https://arxiv.org/e-print/2211.01232


extracting tarball to tmp_2211.01232...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00657-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00657) | **The Close AGN Reference Survey (CARS). A parsec scale multi-phase  outflow in the super-Eddington NLS1 Mrk 1044**  |
|| <mark>Nico Winkel</mark>, et al. -- incl., <mark>Knud Jahnke</mark>, <mark>Miguel A. Pérez-Torres</mark> |
|*Appeared on*| *2022-11-03*|
|*Comments*| *17 pages. 12 figures. Re-submitted to A&A after minor revision. Comments welcome*|
|**Abstract**| The interaction between Active Galactic Nuclei (AGN) and their host galaxies is scarcely resolved. Narrow-line Seyfert 1 (NLS1) galaxies are believed to represent AGN at early stages of their evolution and allow to observe AGN feeding and feedback processes at high accretion rates. We apply a spectroastrometric analysis to VLT MUSE NFM-AO observations of Mrk 1044, a nearby super-Eddington accreting NLS1. This allows us to map two ionised gas outflows traced by [O$\,$III] which have velocities of $-560\pm20\,{\rm km\:s}^{-1}$ and $-144 \pm 5 \,{\rm km\:s}^{-1}$. Both outflows are spatially unresolved and located close to the galaxy nucleus ($<1\,{\rm pc}$). They have gas densities higher than $10^5\,{\rm cm}^{-3}$, which implies that the BPT diagnostic cannot be used to constrain the underlying ionisation mechanism. We explore whether an expanding shell model can describe the velocity structure of Mrk 1044's unresolved multi-phase outflow. A kinematic analysis suggests that significant turbulence may be present in the ISM around the nucleus, which may lead to a condensation rain, potentially explaining the efficient feeding of Mrk 1044's AGN. We identify an additional ionised gas outflowing component that is spatially resolved. It has a velocity of $-211 \pm 22 \,{\rm km\:s}^{-1}$ and a projected size of $4.6 \pm 0.6 \,{\rm pc}$. Within the innermost 0.5" (160$\,{\rm pc}$) around the nucleus we detect modest star formation hidden by the beam-smeared emission from the outflow, which suggests that Mrk 1044's AGN phase set on recently. We estimate that the multi-phase outflow has been launched $<10^4 \,{\rm yrs}$ ago. It carries enough mass and energy to impact the host galaxy star formation on different spatial scales, highlighting the complexity of the AGN feeding and feedback cycle in its early stages. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00667-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00667) | **HD 191939 revisited: New and refined planet mass determinations, and a  new planet in the habitable zone**  |
|| J. Orell-Miquel, et al. -- incl., <mark>J. Livingston</mark> |
|*Appeared on*| *2022-11-03*|
|*Comments*| *Accepted for publication in A&A. 20 pages, 8 figures*|
|**Abstract**| HD 191939 (TOI-1339) is a nearby (d=54pc), bright (V=9mag), and inactive Sun-like star (G9 V) known to host a multi-planet transiting system. Ground-based spectroscopic observations confirmed the planetary nature of the three transiting sub-Neptunes (HD 191939 b, c, and d) originally detected by TESS and were used to measure the masses for planets b and c with 3$\sigma$ precision. These previous observations also reported the discovery of an additional Saturn-mass planet (HD 191939 e) and evidence for a further, very long-period companion (HD 191939 f). Here, we report the discovery of a new non-transiting planet in the system and a refined mass determination of HD 191939 d. The new planet, HD 191939 g, has a minimum mass of 13.5$\pm$2.0 M$_\oplus$ and a period of about 280 d. This period places the planet within the conservative habitable zone of the host star, and near a 1:3 resonance with HD 191939 e. The compilation of 362 radial velocity measurements with a baseline of 677 days from four different high-resolution spectrographs also allowed us to refine the properties of the previously known planets, including a 4.6$\sigma$ mass determination for planet d, for which only a 2$\sigma$ upper limit had been set until now. We confirm the previously suspected low density of HD 191939 d, which makes it an attractive target for attempting atmospheric characterisation. Overall, the planetary system consists of three sub-Neptunes interior to a Saturn-mass and a Uranus-mass planet plus a high-mass long-period companion. This particular configuration has no counterpart in the literature and makes HD 191939 an exceptional multi-planet transiting system with an unusual planet demographic worthy of future observation. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00793-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00793) | **ALMA Observations for CO Emission from Luminous Lyman-break Galaxies at  $z=6.0293$-$6.2037$**  |
|| Yoshiaki Ono, et al. -- incl., <mark>Livia Vallini</mark>, <mark>Chien-Hsiu Lee</mark> |
|*Appeared on*| *2022-11-03*|
|*Comments*| *33 pages, 12 figures, accepted for publication in ApJ*|
|**Abstract**| We present our new Atacama Large Millimeter/submillimeter Array (ALMA) observations targeting CO(6-5) emission from three luminous Lyman break galaxies (LBGs) at $z_{\rm spec} = 6.0293$-$6.2037$ found in the Subaru/Hyper Suprime-Cam survey, whose [OIII]$88\mu$m and [CII]$158\mu$m emission have been detected with ALMA. We find a marginal detection of the CO(6-5) line from one of our LBGs, J0235-0532, at the $\simeq 4 \sigma$ significance level and obtain upper limits for the other two LBGs, J1211-0118 and J0217-0208. Our $z=6$ luminous LBGs are consistent with the previously found correlation between the CO luminosity and the infrared luminosity. The unique ensemble of the multiple far-infrared emission lines and underlying continuum fed to a photodissociation region model reveal that J0235-0532 has a relatively high hydrogen nucleus density that is comparable to those of low-$z$ (U)LIRGs, quasars, and Galactic star-forming regions with high $n_{\rm H}$ values, while the other two LBGs have lower $n_{\rm H}$ consistent with local star-forming galaxies. By carefully taking account of various uncertainties, we obtain total gas mass and gas surface density constraints from their CO luminosity measurements. We find that J0235-0532 locates below the Kennicutt-Schmidt (KS) relation, comparable to the previously CO(2-1) detected $z=5.7$ LBG, HZ10. Combined with previous results for dusty starbursts at similar redshifts, the KS relation at $z=5$-$6$ is on average consistent with the local one. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01044-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01044) | **The discovery of three hot Jupiters, NGTS-23b, 24b and 25b, and updated  parameters for HATS-54b from the Next Generation Transit Survey**  |
|| David G. Jackson, et al. -- incl., <mark>Beth A. Henderson</mark>, <mark>James McCormac</mark>, <mark>Alexis M. S. Smith</mark> |
|*Appeared on*| *2022-11-03*|
|*Comments*| *17 pages, 8 figures. Accepted for publication in MNRAS. arXiv admin note: text overlap with arXiv:2103.10302*|
|**Abstract**| We report the discovery of three new hot Jupiters with the Next Generation Transit Survey (NGTS) as well as updated parameters for HATS-54b, which was independently discovered by NGTS. NGTS-23b, NGTS-24b and NGTS-25b have orbital periods of 4.076, 3.468, and 2.823 days and orbit G-, F- and K-type stars, respectively. NGTS-24 and HATS-54 appear close to transitioning off the main-sequence (if they are not already doing so), and therefore are interesting targets given the observed lack of Hot Jupiters around sub-giant stars. By considering the host star luminosities and the planets' small orbital separations (0.037 - 0.050 au), we find that all four hot Jupiters are above the minimum irradiance threshold for inflation mechanisms to be effective. NGTS-23b has a mass of 0.61 $M_{J}$ and radius of 1.27 $R_{J}$ and is likely inflated. With a radius of 1.21 $R_{J}$ and mass of 0.52 $M_{J}$, NGTS-24b has a radius larger than expected from non-inflated models but its radius is smaller than the predicted radius from current Bayesian inflationary models. Finally, NGTS-25b is intermediate between the inflated and non-inflated cases, having a mass of 0.64 $M_{J}$ and a radius of 1.02 $R_{J}$. The physical processes driving radius inflation remain poorly understood, and by building the sample of hot Jupiters we can aim to identify the additional controlling parameters, such as metallicity and stellar age. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01172-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01172) | **Calculation of rescaling factors and nuclear multiplication of muons in  extensive air showers**  |
|| Kevin Almeida Cheminant, et al. -- incl., <mark>Markus Roth</mark>, <mark>Henryk Wilczyński</mark> |
|*Appeared on*| *2022-11-03*|
|*Comments*| *This work corresponds to the presentation at the ICNFP 2022 at Kolymbari, Crete, in September 2022. The proceedings will be published in Physica Scripta. arXiv admin note: text overlap with arXiv:2108.07527*|
|**Abstract**| Recent results obtained from leading cosmic ray experiments indicate that simulations using LHC-tuned hadronic interaction models underestimate the number of muons in extensive air showers compared to experimental data. This is the so-called muon deficit problem. Determination of the muon component in the air shower is crucial for inferring the mass of the primary particle, which is a key ingredient in the efforts to pinpoint the sources of ultra-high energy cosmic rays.In this paper, we present a new method to derive the muon signal in detectors, which uses the difference between the total reconstructed (data) and simulated signals is roughly independent of the zenith angle, but depends on the mass of the primary cosmic ray. Such a method offers an opportunity not only to test/calibrate the hadronic interaction models, but also to derive the $\beta$ exponent, which describes an increase of the number of muons in a shower as a function of the energy and mass of the primary cosmic ray. Detailed simulations show a dependence of the $\beta$ exponent on hadronic interaction properties, thus the determination of this parameter is important for understanding the muon deficit problem. We validate the method by using Monte Carlo simulations for the EPOS-LHC and QGSJetII-04 hadronic interaction models, and showing that this method allows us to recover the ratio of the muon signal between EPOS-LHC and QGSJetII-04 and the average $\beta$ exponent for the studied system, within less than a few percent. This is a consequence of the good recovery of the muon signal for each primary included in the analysis. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01232-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01232) | **The Luminosity Phase Space of Galactic and Extragalactic X-ray  Transients Out to Intermediate Redshifts**  |
|| Ava Polzin, et al. -- incl., <mark>Deanne Coppejans</mark> |
|*Appeared on*| *2022-11-03*|
|*Comments*| *11 figures, 12 tables; to be submitted to the AAS Journals in ~a week. Comments welcome!*|
|**Abstract**| We present a detailed compilation and analysis of the X-ray phase space of low- to intermediate-redshift transients that consolidates observed light curves (and theory where necessary) for a large variety of classes of transient/variable phenomena in the 0.3--10 keV band including gamma-ray burst afterglows, supernovae, supernova shocks interacting with the environment, tidal disruption events and active galactic nuclei, fast blue optical transients, cataclysmic variables, magnetar flares/outbursts and fast radio bursts, cool stellar flares, X-ray binary outbursts, and ultraluminous X-ray sources. Our over-arching goal is to offer a comprehensive resource for the examination of these ephemeral events, extending the X-ray duration-luminosity phase space (DLPS) to show luminosity evolution. We use existing observations (both targeted and serendipitous) to characterize the behavior of various transient/variable populations. Contextualizing transient signals in the larger DLPS serves two primary purposes: to identify areas of interest (i.e., regions in the parameter space where one would expect detections, but in which observations have historically been lacking) and to provide initial qualitative guidance in classifying newly discovered transient signals. We find that while the most luminous (largely extragalactic) and least luminous (largely Galactic) part of the phase space is well-populated at $t > 0.1$ days, intermediate luminosity phenomena (L$_x = 10^{34} - 10^{42}$ erg s$^{-1}$) represent a gap in the phase space. We thus identify L$_x = 10^{34} - 10^{42}$ erg s$^{-1}$ and $t = 10^{-4} - 0.1$ days as a key discovery phase space in transient X-ray astronomy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00674-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00674) | **Measuring bar pattern speeds from single simulation snapshots**  |
|| <mark>Walter Dehnen</mark>, Marcin Semczuk, Ralph Schoenrich |
|*Appeared on*| *2022-11-03*|
|*Comments*| *7 pages, accepted at MNRAS*|
|**Abstract**| We describe methods to measure simultaneously the orientation angle $\psi$ and pattern speed $\Omega=\mathrm{d}\psi/\mathrm{d}t$ from single snapshots of simulated barred galaxies. Unlike previous attempts, which suffer from systematic errors of 5-25%, our approach is unbiased. It can be extended to obtain the rate and axis of rotation, i.e. the vector $\boldsymbol{\Omega}$. We provide computer code implementing our method. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.01067-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.01067) | **The neutron star population in M28: a joint Chandra/GBT look at pulsar  paradise**  |
|| Eda Vurgun, et al. -- incl., <mark>Manuel Linares</mark> |
|*Appeared on*| *2022-11-03*|
|*Comments*| *26 pages, Accepted for publication in The Astrophysical Journal*|
|**Abstract**| We present the results of a deep study of the neutron star (NS) population in the globular cluster M28 (NGC 6626), using the full 330-ks 2002-2015 ACIS dataset from the Chandra X-ray Observatory and coordinated radio observations taken with the Green Bank Telescope (GBT) in 2015. We investigate the X-ray luminosity (Lx), spectrum, and orbital modulation of the 7 known compact binary millisecond pulsars (MSPs) in the cluster. We report two simultaneous detections of the redback PSR J1824-2452I (M28I) and its X-ray counterpart. We discover a double-peaked X-ray orbital flux modulation in M28I during its pulsar state, centered around pulsar inferior conjunction. We analyze the spectrum of the quiescent neutron star low-mass X-ray binary to constrain its mass and radius. Using both hydrogen and helium NS atmosphere models, we find a NS radius of R = 9.5-11.5 km and R = 13.5 - 16.7 km, respectively, for a neutron star mass of 1.4 Msun. We also search for long-term variability in the 46 brightest X-ray sources and report the discovery of six new variable low luminosity X-ray sources in M28. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.00657.md
    + _build/html/tmp_2211.00657/./Figures/05_Appendix/shell_model_parameters.png
    + _build/html/tmp_2211.00657/./Figures/03_Analysis/Mrk1044_QSO_spec_full.png
    + _build/html/tmp_2211.00657/./Figures/03_Analysis/spectroastrometry.png
exported in  _build/html/2211.00667.md
    + _build/html/tmp_2211.00667/./Figs/R-M_DIAGRAM_referee.png
    + _build/html/tmp_2211.00667/./Figs/RVS_ALL_DATASETS_no_grid.png
    + _build/html/tmp_2211.00667/./Figs/TESS_PHOTOMETRY_jf_0_referee.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# The Close AGN Reference Survey (CARS): A parsec scale multi-phase outflow in the super-Eddington NLS1 Mrk 1044

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.00657-b31b1b.svg)](https://arxiv.org/abs/2211.00657)<mark>Appeared on: 2022-11-03</mark> - _17 pages. 12 figures. Re-submitted to A&A after minor revision. Comments welcome_

</div>
<div id="authors">

<mark><mark>Nico Winkel</mark></mark>, et al. -- incl., <mark><mark>Knud Jahnke</mark></mark>, <mark><mark>Miguel A. Pérez-Torres</mark></mark>

</div>
<div id="abstract">

**Abstract:** The interaction between Active Galactic Nuclei (AGN) and their host galaxies is scarcely resolved. Narrow-line Seyfert 1 (NLS1) galaxies are believed to represent AGN at early stages of their evolution and allow to observe AGN feeding and feedback processes at high accretion rates.We aim to constrain the properties of the ionised gas outflow in Mrk 1044, a nearby super-Eddington accreting NLS1. Based on the outflow energetics and the associated timescales, we estimate the outflow's future impact on the ongoing host galaxy star formation on different spatial scales.We apply a spectroastrometric analysis to VLT MUSE NFM-AO observations of Mrk 1044's nucleus. This allows us to map two ionised gas outflows traced by[$\ion${O}{iii}]which have velocities of$-560 \pm 20 {\rm km\:s}^{-1}$and$-144 \pm 5  {\rm km\:s}^{-1}$.   Furthermore, we use an archival HST/STIS spectrum to identify two Ly-$\alpha$absorbing components that escape from the centre with approximately twice the velocity of the ionised gas components.Both[$\ion${O}{iii}]outflows are spatially unresolved and located close to the AGN ($< 1 {\rm pc}$). They have gas densities higher than$10^5  {\rm cm}^{-3}$, which implies that the BPT diagnostic cannot be used to constrain the underlying ionisation mechanism.   We explore whether an expanding shell model can describe the velocity structure of Mrk 1044's multi-phase outflow.   We find an additional ionised gas outflowing component that is spatially resolved. It has a velocity of$-211 \pm 22  {\rm km\:s}^{-1}$and projected size of\mbox{$4.6 \pm 0.6  {\rm pc}$}.   A kinematic analysis suggests that significant turbulence may be present in the ISM around the nucleus, which may lead to a condensation rain, potentially explaining the efficient feeding of Mrk 1044's AGN.   Within the innermost 0.5$\arcsec$(160$ {\rm pc}$) we detect modest star formation hidden by the beam-smeared emission from the outflow.We estimate that the multi-phase outflow has been launched$< 10^4  {\rm yrs}$ago. Together with the star formation in the vicinity of the nucleus, this suggests that Mrk 1044's AGN phase set on recently.    The outflow carries enough mass and energy to impact the host galaxy star formation on different spatial scales, highlighting the complexity of the AGN feeding and feedback cycle in its early stages.

</div>

<div id="div_fig1">

<img src="tmp_2211.00657/./Figures/05_Appendix/shell_model_parameters.png" alt="Fig7" width="100%"/>

**Figure 7. -** Dependence of the emission line flux maximum corresponding velocity on inclination. Independent of how concentrated the light profile is ($\alpha/\theta_0$), the geometric model requires high inclinations of the [$\ion${O}{iii}]-emitting shell to significantly reduce the peak velocity by a factor of approximately two.
  (*fig:shell_model_parameters*)

</div>
<div id="div_fig2">

<img src="tmp_2211.00657/./Figures/03_Analysis/Mrk1044_QSO_spec_full.png" alt="Fig8" width="100%"/>

**Figure 8. -** Modelling Mrk 1044's AGN spectrum in the H$\beta$-[$\ion${O}{iii}] region (left) and H$\alpha$-[$\ion${N}{ii}] region (right). To reproduce the observed spectrum (black) we use a multi-Gaussian model for H$\beta$(blue), [$\ion${O}{iii}](green), $\ion${Fe}{ii}(purple), H$\alpha$(blue, right panel) and [$\ion${N}{ii}](red). The best-fit spectrum is shown as a red line and well reproduces the prominent emission lines, including the blue shoulder of the [$\ion${O}{iii}] narrow line. (*fig:MUSE_fit*)

</div>
<div id="div_fig3">

<img src="tmp_2211.00657/./Figures/03_Analysis/spectroastrometry.png" alt="Fig9" width="100%"/>

**Figure 9. -** Spectroastrometric analysis of Mrk 1044's central region using the original MUSE NFM-AO data cube.
            The left panels show an arbitrary example spectrum that is picked from the small window around the nucleus shown in the panels on the right. To fit the spectrum, we keep the kinematics and line ratios of the kinematics fixed to that of the components found in the AGN spectrum.
            The panels on the right show the surface brightness within the central 150$ {\rm mas}$ for the kinematic components where the flux maps are normalised to their peak flux.
            Here we only show the measure light distribution from the BLR H$\beta$(left), [$\ion${O}{iii}]-wing (middle), and narrow [$\ion${O}{iii}]-core (right) component. From top to bottom the maps show the 2D light distributions, the best-fit model of the PSF and the residual maps. The red cross indicates the PSFAO19 centroid for the best-fit PSFAO19 fit to the BLR H$\beta$. The green and white crosses indicate the centroids for the PSFAO19 models to the surface brightness maps of the [$\ion${O}{iii}]-wing and core respectively, in which the amplitude and position of the Moffat model were varied only. The offset of both [$\ion${O}{iii}] core and wing component from the AGN position are smaller than $0.1 {\rm px}$. (*fig:spectroastrometry_maps*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# HD 191939 revisited: New and refined planet mass determinations, and a new planet in the habitable zone.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.00667-b31b1b.svg)](https://arxiv.org/abs/2211.00667)<mark>Appeared on: 2022-11-03</mark> - _Accepted for publication in A&A. 20 pages, 8 figures_

</div>
<div id="authors">

J. Orell-Miquel, et al. -- incl., <mark>J. Livingston</mark>

</div>
<div id="abstract">

**Abstract:** HD 191939 (TOI-1339) is a nearby (d$=$54 pc), bright (V$=$9 mag), and inactive Sun-like star (G9 V) known to host a multi-planet transiting system. Ground-based spectroscopic observations confirmed the planetary nature of the three transiting sub-Neptunes (HD 191939 b, c, and d) originally detected by\textit{TESS}and were used to measure the masses for planets b and c with$3\sigma$precision. These previous observations also reported the discovery of an additional Saturn-mass planet (HD 191939 e) and evidence for a further, very long-period companion (HD 191939 f). Here, we report the discovery of a new non-transiting planet in the system and a refined mass determination of HD 191939 d. The new planet, HD 191939 g, has a minimum mass of 13.5$\pm$2.0$M_\oplus$and a period of about 280 d. This period places the planet within the conservative habitable zone of the host star, and near a 1:3 resonance with HD 191939 e.  The compilation of 362 radial velocity measurements with a baseline of 677 days from four different high-resolution spectrographs also allowed us to refine the properties of the previously known planets, including a$4.6\sigma$mass determination for planet d, for which only a$2\sigma$upper limit had been set until now. We confirm the previously suspected low density of HD 191939 d, which makes it an attractive target for attempting atmospheric characterisation.  Overall, the planetary system consists of three sub-Neptunes interior to a Saturn-mass and a Uranus-mass planet plus a high-mass long-period companion. This particular configuration has no counterpart in the literature and makes HD 191939 an exceptional multi-planet transiting system with an unusual planet demographic worthy of future observation.

</div>

<div id="div_fig1">

<img src="tmp_2211.00667/./Figs/R-M_DIAGRAM_referee.png" alt="Fig2" width="100%"/>

**Figure 2. -** \label{fig: SYSTEM DIAGRAM PLOT}
    Mass--period (\textit{top panel}) and mass--radius (\textit{bottom panel}) diagrams for well-characterised planets with masses and radii measured with a precision better than 30\%, from 1 ${M}_\oplus$ up to 1.5 ${M}_{\mathrm{J}}$ and 1 ${R}_\oplus$ up to 1.5 ${R}_{\mathrm{J}}$, from the TEPCat database (February 2022; \citealp{Southworth_database}) and \url{http://exoplanet.eu}. HD 191939 planets are colour coded and marked with a filled circle with error bars (the radii  of HD 191939 e and g are forecasted). Vertical colour bands indicate the $\pm 1\sigma$ mass regions of HD 191939 g and e. Temperate planets with $T_{\mathrm{eq}}$ = 250--395 K are marked by blue squares. The mass--radius panel also shows theoretical composition models at 300 K from \citet{Zeng_models}. (*fig: SYSTEM DIAGRAM PLOT*)

</div>
<div id="div_fig2">

<img src="tmp_2211.00667/./Figs/RVS_ALL_DATASETS_no_grid.png" alt="Fig8" width="100%"/>

**Figure 8. -** \label{fig: RV DATA}
Time series of RV measurements taken by APF (blue circles), HIRES (green up triangles), CARMENES (red squares), and HARPS-N (orange down triangles). (*fig: RV DATA*)

</div>
<div id="div_fig3">

<img src="tmp_2211.00667/./Figs/TESS_PHOTOMETRY_jf_0_referee.png" alt="Fig14" width="100%"/>

**Figure 14. -** \label{fig: TESS SECTORS}\textit{TESS} photometry from Sectors 15--19, 21, 22, 24, and 25 along with the best-fit model (see Fig. \ref{fig: TESS SECTORS 41 48} for Sectors 41 and 48). Upward-pointing triangles mark the transits for HD 191939 b (red), c (cyan), and d (green). Downward-pointing triangles with error bars mark the expected $t_0$ and $\pm   1\sigma$ uncertainty for the non-transiting planets HD 191939 e (magenta) and g (orange). (*fig: TESS SECTORS*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

80  publications in the last 7 days.
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers